In [25]:
import cv2
import numpy as np
import serial
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [26]:
cam = 2

model_version = "my_model_14.0.keras"
Im_Width = 128
Im_Height = 128

In [27]:
def crop_image(frame, x, y, w, h):
    return frame[y:y+h, x:x+w]

In [28]:
def measure_brightness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    brightness = np.mean(gray)
    return brightness 

In [29]:
def measure_edge_richness(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    edges = cv2.Canny(gray, 200, 400)  # Apply Canny edge detection
    edge_richness = np.sum(edges)  # Sum of edge pixel intensities
    return edge_richness

In [30]:
def select_best_frame(frame_buffer):

    best_frame = None
    highest_brightness = 100
    highest_edge_richness = 100000

    for frame in frame_buffer:

        #Crop
        cropped_frame = crop_image(frame, 220, 0, 200, 200)

        # Measure brightness
        brightness = measure_brightness(cropped_frame)

        if brightness > 100:
            # Update the best frame if this one is better
            if brightness > highest_brightness:
                highest_brightness = brightness
                best_frame = frame  # Save the frame as the best frame
    
        elif brightness > 50 and brightness < 100:
            # Apply edge detection
            edge_richness = measure_edge_richness(cropped_frame)
            # Update the best frame if this one is better
            if edge_richness > highest_edge_richness:
                highest_edge_richness = edge_richness
                best_frame = frame  # Save the cropped frame as the best frame

    return best_frame  # Returns None if no frames meet the threshold


In [31]:
def process_frame(frame, target_size):

    # Resize the frame to the target size
    resized_frame = cv2.resize(frame, target_size)

    # Normalize pixel values to [0, 1]
    normalized_frame = resized_frame / 255.0

    # Convert to NumPy array and add batch dimension
    frame_array = img_to_array(normalized_frame)
    frame_array = np.expand_dims(frame_array, axis=0)  # Shape: (1, height, width, channels)

    return frame_array

In [32]:
def setup_serial_connection(port='COM4', baud_rate=9600):
    arduino = serial.Serial(port, baud_rate, timeout=1)
    print("Arduino connected. Waiting for initialization...")
    arduino.flush()  # Clear any remaining data
    return arduino

In [33]:
def send_to_arduino(arduino, value):
    arduino.write(f"{value}\n".encode())
    print(f"Sent: {value}")

In [34]:
def predict_frame(frame, model, target_size):

    # Preprocess the frame
    frame_array = process_frame(frame, target_size)

   # Make a prediction
    prediction = model.predict(frame_array)[0][0]  # Get the confidence score
    print(f"Raw Prediction: {prediction}")

    # Interpret the prediction
    if prediction > 0.5:
        send_to_arduino(arduino, 0)
        return "Non-Defective", prediction
        
    else:
        send_to_arduino(arduino, 1)
        return "Defective", prediction


In [35]:
def capture(cam, model_path, target_size=(Im_Width, Im_Height)):

    cap = cv2.VideoCapture(cam)  # Open webcam
    frame_buffer = []
    max_frames = 10  # Number of frames to evaluate

    # Load the model
    print("Loading model...") 
    model = load_model(model_path)
    print("Model loaded successfully.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame. Exiting...")
            break

        frame_buffer.append(frame)

        # Process the buffer when it's full
        if len(frame_buffer) == max_frames:
            best_frame = select_best_frame(frame_buffer)
            frame_buffer = []  # Clear the buffer

            # Process and predict
            if best_frame is not None:
                feedback, confidence = predict_frame(best_frame, model, target_size)
                print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

                prob = abs(round(confidence,2)- 0.5)*200

                # Display feedback on the frame
                cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f"Confidence: {prob:.0f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Display the best frame
                cv2.imshow("Best Frame", best_frame)

            else:
                # Load and display the "no-bottle" image in the same window
                image_path = "no-bottle.bmp"
                image = cv2.imread(image_path)

                if image is None:
                    print("Error: Could not load 'no-bottle' image.")
                else:
                    cv2.putText(image, "No Bottle!", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                    cv2.imshow("Best Frame", image)
                    

        # Exit on 'ESC' key
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


In [36]:
if __name__ == "__main__":
    model_path = model_version  # Path to your saved model
    arduino = setup_serial_connection('COM4')  # Connect once
    capture(cam, model_path, target_size=(Im_Width, Im_Height))

Arduino connected. Waiting for initialization...
Loading model...
Model loaded successfully.
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Raw Prediction: 0.03271273151040077
Sent: 1
Prediction: Defective (Confidence: 0.03)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Raw Prediction: 0.2578921616077423
Sent: 1
Prediction: Defective (Confidence: 0.26)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Raw Prediction: 0.00858923140913248
Sent: 1
Prediction: Defective (Confidence: 0.01)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Raw Prediction: 0.00044687901390716434
Sent: 1
Prediction: Defective (Confidence: 0.00)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Raw Prediction: 0.1262766718864441
Sent: 1
Prediction: Defective (Confidence: 0.13)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Raw Prediction: 0.042168281972408295
Sent: 1
Prediction: Defective (Confidence: 0.04)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Raw Prediction: 0.2455901950597763
Sent: 1
Prediction: Defective (Confidence: 0.25)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
Raw Predi